In [1]:
import sys
import numpy as np
from Bio import PDB
import pandas as pd
import seaborn as sns
import os
import pickle
import random
import torch
import esm
import re
import math
from Bio import SeqIO

from utils import old_aa, position_aa, new_aa,Esm2_embedding ,Create_mut_sequence_multiple, dataset_builder

2.4.1+cu118


In [3]:
#SELEZIONA DATASET

dataset_mutations = pd.read_csv('../DeltaDelta_BELLO/ptmul-NR.tsv', sep = '\t')
dataset_mutations = dataset_mutations.rename(columns={'PDB':'ID'})
# Lista per salvare le sequenze
records = []

# Leggere il file FASTA
#fasta_file = "S669_castrense.fasta.txt"  # Cambia con il nome del tuo file
fasta_file = "../DeltaDelta_BELLO/ptmul-NR.fasta.txt"  # Cambia con il nome del tuo file

for record in SeqIO.parse(fasta_file, "fasta"):
    records.append([record.id, str(record.seq)])  # ID e Sequenza

# Creare il DataFrame
dataset_sequences = pd.DataFrame(records, columns=["ID", "Sequence"])
dataset_sequences.head()

print(dataset_mutations.head())    
print(dataset_sequences.head())

df = pd.read_csv('../DeltaDelta_BELLO/ptmul-NR.tsv', sep = '\t')
df.rename(columns={'PDB':'ID'},inplace=True)
df['Sequence'] = df['ID'].map(lambda x: dataset_sequences[dataset_sequences['ID']==x]['Sequence'].item())
df

       ID          MTS    DDG
0  P29957   Q82C_A123C  1.870
1  P09850  V126C_A180C -7.525
2  P09850  S128C_N176C  1.670
3  P00094    K35E_K53E -0.800
4  P02417     F5V_K12M -2.035
       ID                                           Sequence
0  P00004  MGDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPG...
1  P00044  MTEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHS...
2  P00094  MKISLTAATVAALVLAAPAFAGDAAKGEKEFNKCKTCHSIIAPDGT...
3  P00099  MKPYALLSLLATGTLLAQGAWAEDPEVLFKNKGCVACHAIDTKMVG...
4  P01236  MNIKGSPWKGSLLLLLVSNLLLCQSVAPLPICPGGAARCQVTLRDL...


In [20]:
df = pd.read_csv('../DeltaDelta_BELLO/S669_castrense.tsv',sep= '\t')
df['MTS']=df.apply(lambda x: x['WT']+str(x['POS'])+x['MT'], axis=1)
df.rename(columns={'PDB':'ID'}, inplace=True)
df = df[['ID', 'MTS', 'DDG']]

#CAMBIA SEGNO SU DATASET CASTRENSE PERCHè SBAGLIATO
df.loc[(df['ID'] == 'P81662'), 'DDG'] = -df.loc[(df['ID'] == 'P81662'), 'DDG']
df.loc[(df['ID'] == 'P51608'), 'DDG'] = -df.loc[(df['ID'] == 'P51608'), 'DDG']


fasta_file = "../DeltaDelta_BELLO/S669_castrense.fasta.txt"
records = []

for record in SeqIO.parse(fasta_file, "fasta"):
    records.append([record.id, str(record.seq)])  # ID e Sequenza

# Creare il DataFrame
dataset_sequences = pd.DataFrame(records, columns=["ID", "Sequence"])
dataset_sequences.head()

#print(dataset_mutations.head())    
print(dataset_sequences.head())
df['Sequence'] = df['ID'].map(lambda x: dataset_sequences[dataset_sequences['ID'] == x.split('WT')[0]]['Sequence'].iloc[0])
df

       ID                                           Sequence
0   1fh5H  SGGGLVKPAGSLKLSCAASGFTFSSYYMYWVRQTPDKRLEWVATIS...
1   1iv7B  GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
2   1iv9A  GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
3  A2IC68  MGWSVNAWSFACKTANGTAIPIGGGSANVYVNLAPVVNVGQNLVVD...
4  E0WVU4  MPLDETNNESYRYLRSVGNTWKFNVEDVHPKMLERLYKRFDTFDLD...


,ID,MTS,DDG,Sequence
0,P0A9D2,S11A,-1.800,MKLFYKPGACSLASHITLRESGKDFTLVSVDLMKKRLENGDDYFAV...
1,P00149,A125H,-2.690,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
2,P00149,A87H,-1.980,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
3,P00149,A112H,-1.700,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
4,P00149,D24H,-1.360,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
...,...,...,...,...
664,1fh5H,S15N,0.600,SGGGLVKPAGSLKLSCAASGFTFSSYYMYWVRQTPDKRLEWVATIS...
665,1iv7B,A73C,0.500,GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
666,1iv7B,F89C,-2.500,GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
667,1iv7B,L62C,-1.900,GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...


In [21]:
#df.to_csv('Data/s669_to_process.csv',index=False)

In [3]:
## S_TRANSITIVE
#assert False
df = pd.read_csv('../git_JANUS_DDG/Data/Stransitive_to_process.csv')

In [4]:
#CARICAMENTO DEL DATASET 

#IL DATASET CARICATO DEVE AVERE LE SEGUENTI COLONNE:

#ID	MTS SEQ
#ID: NOME UNIVOCO PER SEQUENZA WILD TYPE
#MTS: MUTS SEP BY '_' (position start by 1)
#Sequence: SEQUENZA LETTERE AA DELLA PROTEINA

#ESEMPIO: P29957 Q82C_A123C_.... MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...

#df = pd.read_csv('../Data/')

dataset_mutations = df[['ID','MTS','DDG','Sequence']].copy()
print(dataset_mutations.head())

dataset_sequences = df.loc[:,['ID','Sequence']].copy()
dataset_sequences = dataset_sequences.drop_duplicates(subset='ID')
print(dataset_sequences.head())

   ID    MTS  DDG                                           Sequence
0  1F  C180Y -2.3  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
1  1R  Y180C  2.3  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
2  2F  C180F -1.6  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
3  2R  F180C  1.6  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
4  3F  C180W -3.9  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
   ID                                           Sequence
0  1F  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
1  1R  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
2  2F  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
3  2R  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
4  3F  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...


In [5]:
dataset_mutations['Old_AA'] = dataset_mutations.apply(old_aa, axis = 1)
dataset_mutations['Pos_AA'] = dataset_mutations.apply(position_aa, axis = 1)
dataset_mutations['New_AA'] = dataset_mutations.apply(new_aa, axis = 1)
dataset_mutations['Pos_AA'] = dataset_mutations['Pos_AA'].map(lambda x: [i-1 for i in x])

print(dataset_mutations.head())
print(dataset_sequences.head())


   ID    MTS  DDG                                           Sequence Old_AA  \
0  1F  C180Y -2.3  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...    [C]   
1  1R  Y180C  2.3  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...    [Y]   
2  2F  C180F -1.6  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...    [C]   
3  2R  F180C  1.6  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...    [F]   
4  3F  C180W -3.9  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...    [C]   

  Pos_AA New_AA  
0  [179]    [Y]  
1  [179]    [C]  
2  [179]    [F]  
3  [179]    [C]  
4  [179]    [W]  
   ID                                           Sequence
0  1F  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
1  1R  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
2  2F  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
3  2R  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...
4  3F  MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...


In [6]:
dataset_multi = dataset_builder(dataset_mutations, dataset_sequences,)

In [ ]:
assert False

In [8]:
import pickle
with open("../DeltaDelta_BELLO/S_transitive.pkl", "wb") as f:
    pickle.dump(dataset_multi, f)

In [9]:
pd.read_csv('Data/s669_to_process.csv')

,ID,MTS,Sequence
0,P0A9D2,S11A,MKLFYKPGACSLASHITLRESGKDFTLVSVDLMKKRLENGDDYFAV...
1,P00149,A125H,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
2,P00149,A87H,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
3,P00149,A112H,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
4,P00149,D24H,MKLRIATIAGLVVLGSGFAVAQTDVIAQRKAILKQMGEATKPIAAM...
...,...,...,...
664,1fh5H,S15N,SGGGLVKPAGSLKLSCAASGFTFSSYYMYWVRQTPDKRLEWVATIS...
665,1iv7B,A73C,GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
666,1iv7B,F89C,GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
667,1iv7B,L62C,GEWEIIDIGPFTQNLGKFAVDEENKIGQYGRLTFNKVIRPCMKKTI...
